# Web Mining and Applied NLP (44-620)

## Requests, JSON, and NLP

### Student Name: Alvaro Quintero Gonzalez


# Project Objectives
This exercise illustrates how to access web-hosted APIs, get back a response in JSONLinks to an external site. format, and extract the information we need from the JSON. Accessing APIs is a key skill for data analysts. We can use web APIs to get stock data, weather data, and much more. We'll use an API to access song lyrics or poems in this exercise. We don't care which API - there are many and they change. The skill skills are being able to (a) make an API request and (b) find the information we need in the returned response. 

In [29]:
import requests
import json
import spacy
import pickle
from spacytextblob.spacytextblob import SpacyTextBlob

print('All prereqs installed')
!pip list

All prereqs installed
Package            Version
------------------ ------------
annotated-types    0.7.0
appnope            0.1.4
asttokens          2.4.1
blis               1.0.1
catalogue          2.0.10
certifi            2024.8.30
charset-normalizer 3.4.0
click              8.1.7
cloudpathlib       0.20.0
comm               0.2.2
confection         0.1.5
cymem              2.0.8
debugpy            1.8.8
decorator          5.1.1
en_core_web_sm     3.8.0
executing          2.1.0
idna               3.10
ipykernel          6.29.5
ipython            8.29.0
jedi               0.19.1
Jinja2             3.1.4
joblib             1.4.2
jupyter_client     8.6.3
jupyter_core       5.7.2
langcodes          3.4.1
language_data      1.2.0
marisa-trie        1.2.1
markdown-it-py     3.0.0
MarkupSafe         3.0.2
matplotlib-inline  0.1.7
mdurl              0.1.2
murmurhash         1.0.10
nest-asyncio       1.6.0
nltk               3.9.1
numpy              2.0.2
packaging          24.2
parso      

1. The following code accesses the [lyrics.ovh](https://lyricsovh.docs.apiary.io/#reference/0/lyrics-of-a-song/search) public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

In [30]:
import requests
import json

# Accessing the API and storing the result
result = json.loads(requests.get('https://api.lyrics.ovh/v1/They Might Be Giants/Birdhouse in your soul').text)

# Write the resulting JSON to a file
with open('lyrics.json', 'w') as json_file:
    json.dump(result, json_file, indent=4)  # Writing with pretty-print
print("Lyrics saved to lyrics.json.")


Lyrics saved to lyrics.json.


2. Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis.  Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotaion?  Answer this question in a comment in your code cell.

In [31]:
import json
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

# Load spaCy model
nlp_model = spacy.load('en_core_web_sm')

# Add sentiment analysis pipeline
nlp_model.add_pipe("spacytextblob", last=True)

# Open and load the JSON file
with open('lyrics.json', 'r') as file:
    song_data = json.load(file)

# Retrieve lyrics from dictionary
song_lyrics = song_data.get('lyrics', '')

# Perform sentiment analysis
doc_analysis = nlp_model(song_lyrics)
sentiment_score = doc_analysis._.blob.polarity  # Get the polarity

# Display the lyrics and the sentiment score
print("Song Lyrics:\n", song_lyrics)
print("\nSentiment Score:", sentiment_score)

# Interpret the sentiment
if sentiment_score > 0:
    print("The lyrics are positive.")
elif sentiment_score < 0:
    print("The lyrics are negative.")
else:
    print("The lyrics are neutral.")


Song Lyrics:
 I'm your only friend 
I'm not your only friend 
But I'm a little glowing friend 
But really I'm not actually your friend 
But I am 


Blue canary in the outlet by the light switch 

Who watches over you 

Make a little birdhouse in your soul 

Not to put too fine a point on it 

Say I'm the only bee in your bonnet 

Make a little birdhouse in your soul 



I have a secret to tell 

From my electrical well 

It's a simple message and I'm leaving out the whistles and bells 

So the room must listen to me 

Filibuster vigilantly 

My name is blue canary one note* spelled l-i-t-e 

My story's infinite 

Like the Longines Symphonette it doesn't rest 



Blue canary in the outlet by the light switch 

Who watches over you 

Make a little birdhouse in your soul 

Not to put too fine a point on it 

Say I'm the only bee in your bonnet 

Make a little birdhouse in your soul 



I'm your only friend 

I'm not your only friend 

But I'm a little glowing friend 

But really I'm not a

3. Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [32]:
import requests
import json

def save_song_lyrics(artist, song, filename):
   
    # Format the URL with the artist and song title
    url = f'https://api.lyrics.ovh/v1/{artist}/{song}'
    response = requests.get(url)
    
    if response.status_code == 200:
        # Parse the JSON response
        lyrics_data = response.json()
        
        # Write the lyrics to the specified file
        with open(filename, 'w') as file:
            json.dump(lyrics_data, file, indent=4)
        
        print(f"Lyrics for '{song}' by {artist} saved to {filename}")
    else:
        print(f"Failed to retrieve lyrics for '{song}' by {artist}. Status code: {response.status_code}")

# Test the function with specified songs
if __name__ == "__main__":
    save_song_lyrics("Justin Bieber", "Baby", "baby_lyrics.json")
    save_song_lyrics("Brian McKnight", "Back At One", "back_at_one_lyrics.json")
    save_song_lyrics("Maverick City Music", "Promises", "promises_lyrics.json")
    save_song_lyrics("Rihanna", "Umbrella", "umbrella_lyrics.json")




Lyrics for 'Baby' by Justin Bieber saved to baby_lyrics.json
Lyrics for 'Back At One' by Brian McKnight saved to back_at_one_lyrics.json
Lyrics for 'Promises' by Maverick City Music saved to promises_lyrics.json
Lyrics for 'Umbrella' by Rihanna saved to umbrella_lyrics.json


4. Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.

In [33]:
import json
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

# Load spaCy model and SpacyTextBlob
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("spacytextblob")

def analyze_lyrics_sentiment(filename, song_name):
    # Load the lyrics from JSON
    with open(filename, 'r') as file:
        lyrics_data = json.load(file)
    
    # Retrieve the lyrics text
    lyrics = lyrics_data.get('lyrics', '')
    
    # Perform sentiment analysis
    doc = nlp(lyrics)
    polarity_score = doc._.blob.polarity
    
    # Print the song name and its polarity score
    print(f"Polarity Score for '{song_name}': {polarity_score}")
    return polarity_score

# Test the function with 3 JSON files
analyze_lyrics_sentiment("baby_lyrics.json", "Baby")
analyze_lyrics_sentiment( "back_at_one_lyrics.json", "Back At One")
analyze_lyrics_sentiment("promises_lyrics.json", "Promises")


Polarity Score for 'Baby': 0.14945200302343165
Polarity Score for 'Back At One': 0.10775027056277058
Polarity Score for 'Promises': 0.16223544973544965


0.16223544973544965

The reported polarity scores generally indicate a slightly positive sentiment for all three songs.- **'Baby'** has a polarity score of 0.149, suggesting it contains light positivity, which aligns with the upbeat and catchy nature of the song that talks about young love and affection.- **'Back At One'** has a lower score of 0.108, indicating a gentle positivity. The lyrics convey themes of love and commitment but with a more serious undertone, which might explain the lower score compared to 'Baby'.- **'Promises'** displays a score of 0.162, suggesting optimism and hopefulness. This aligns with the song's theme of faithfulness and assurance, thus reflecting a moderately positive sentiment.